In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap
import seaborn as sns
import pandas as pd

import pickle
# 
from tqdm import tqdm

import sleap


import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

import numba
from numba import jit
import parmap


from Imputation import Impute, CentreBody

from Imputation import *

Autosaving every 180 seconds


In [2]:
#####################################################
#####################################################
#####################################################
def median_filter(data, max_gap, filter_width):
    
    for f in trange(data.shape[1]):
        for l in range(data.shape[2]):
            x = data[:,f,l]
            for k in range(1000):
                idx = np.where(np.isnan(x))[0]
                if idx.shape[0]==0:
                    break

                if idx[0]==0:
                    idx=idx[1:]
                x[idx] = x[idx-1]

            x = scipy.ndimage.median_filter(x, size=filter_width)
            data[:,f,l]= x

    return data

#
def get_durations(data, ctr, animal_id, min_duration = 25, plotting = False):
    clrs = ['black','green', 'magenta','brown']
    labels = ['two', 'all body', 'headnose', 'sliding']

    
    #
    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    d = np.load(root_dir + '/animalID_'+
                str(animal_id)+
                '_alldata_imputed.npy')
    print ("animal: ", animal_id, "all data imputed: ", d.shape)

    #
    starts = []
    ends = []
    starts.append(data[0])
    for d in range(1,data.shape[0],1):
        if (data[d]-data[d-1])>1:
            ends.append(data[d-1])
            starts.append(data[d])

    #
    starts = np.array(starts)
    ends = np.array(ends)
    if starts.shape[0]==ends.shape[0]+1:
        starts=starts[:-1]

    durations = (ends - starts)+1
    
    dd = []
    vectors = []
    for k in range(durations.shape[0]):
        temp = durations[k]
        if temp>=min_duration:
            for p in range(0,temp-min_duration,1):
                dd.append(min_duration)
                
                #print (starts[k]+p, ends[k]+p)
                vectors.append([starts[k]+p, starts[k]+min_duration+p])

    vectors = np.vstack(vectors)
    
    
    #
    if plotting:
        width = 10
        bins = np.arange(0,250,width)
        y = np.histogram(durations, bins= bins)
        plt.plot(y[1][1:], y[0], label=labels[ctr], c=clrs[ctr])

        # also plot sliding window data
        y = np.histogram(dd, bins= bins)
        plt.plot(y[1][1:], y[0], label=labels[-4], c=clrs[-4])

            #
        plt.semilogy()
        plt.plot([25,25],[0,5000],'r--', label='1sec')
        plt.plot([12.5,12.5],[0,5000],'b--', label='0.5sec')
        plt.ylim(bottom=1)
        plt.title("Cohort 1, March 26, 23hrs;  animal_id " + str(animal_id))
        plt.legend()

    return durations, dd, vectors
    
#
def get_lengths(animal_id):

    #
    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    d = np.load(root_dir + '/animalID_'+
                str(animal_id)+
                '_alldata_imputed.npy')

    fname_out = os.path.join(root_dir,"animalID_"+str(animal_id)+"lengths.npz")
    
    if os.path.exists(fname_out)==False:

        # 
        two = []
        six = []
        
        headnose = []
        for k in trange(d.shape[0]):
            temp = d[k]
            idx1 = np.where(np.isnan(temp[:,0])==False)[0]
            idx2 = np.where(np.isnan(temp[:,1])==False)[0]
            
            # make sure sme missing
            if idx1.shape[0]!=idx2.shape[0]:
                continue

            # 
            if idx1.shape[0]==6:
                six.append(k)
                continue

            # 
            if idx1.shape[0]>=2:
                two.append(k)
                if idx1[0]==0 and idx1[1]==1:
                    headnose.append(k)

        # 
        two = np.array(two)
        six = np.array(six)
        headnose = np.array(headnose)
        print ("two: ", two.shape, " six: ", six.shape)

        np.savez(fname_out,
                two=two,
                six=six,
                headnose=headnose)
    else:
        d = np.load(fname_out, allow_pickle=True)
        two = d['two']
        six = d['six']
        headnose = d['headnose']
        
    return two, six, headnose

# 
def visualize_durations():
    fig=plt.figure()
    animal_ids = np.arange(4)
    clrs = ['red','blue', 'cyan','green']
    labels = ['two', 'all body', 'headnose', 'sliding']

    two_feat_flag = False
    min_durations = np.array([25,50,125,250,500,750,1500,3000,4500,6000,7500])
    for animal_id in animal_ids:

        #
        # max_gap = 10
        # filter_width = 25
        # d = median_filter(d, max_gap, filter_width)

        #
        two, six, headnose = get_lengths(animal_id)

        res = []
        for min_duration in min_durations:
            # 
            if two_feat_flag:
                _, dur_1sec, _ = get_durations(two, 0, animal_id, min_duration = min_duration, plotting = False)
            else:
                _, dur_1sec, _ = get_durations(six, 0, animal_id, min_duration = min_duration, plotting = False)

            print (animal_id, len(dur_1sec))
            res.append(len(dur_1sec))

        #
        plt.plot(min_durations/25., res, 
                 color=clrs[animal_id],
                 linewidth=3, label = "animal: "+str(animal_id))

        #
        #break
    plt.ylim(bottom=1)
    #plt.xlim(0,7500/25)
    plt.xlim(0,1500/25)
    #plt.suptitle("# of segments of specific duration containing head + nose features",fontsize=20)
    plt.suptitle("# of segments of specific duration containing all six features",fontsize=20)
    #plt.semilogy()
    plt.xlabel("Duration of segment (Sec)", fontsize=20)
    plt.ylabel("# of segments", fontsize=20)
    plt.legend()
    plt.show()




#
def centre_and_align2_pairwise(data, centre_pt = 0, rotation_pt = 1):

    # centre the data on the nose
    # data[:,0] -= data[centre_pt,0]
    # data[:,1] -= data[centre_pt,1]
    #print ("data; ", data.shape)

    translation_pt = data[0,centre_pt]
    data -= translation_pt
    

    # get angle between +x axis and head location (i.e. 2nd position)
    # 2ND POSITION ALIGNMENT
    angle = -np.arctan2(*data[0,rotation_pt].T[::-1])-np.pi/2

    #print ("agnel: ", angle, "  translation pt: ", translation_pt, '  Rotation pt: ', rotation_pt)
    
    # get rotation
    rotmat = np.array([[np.cos(angle), -np.sin(angle)],
                       [np.sin(angle),  np.cos(angle)]])

    # Apply rotation to each row of m
    data_rot = []
    for p in range(data.shape[0]):
        data_rot.append((rotmat @ data[p].T).T)

    data_rot = np.array(data_rot)
    
    idx = np.where(np.isnan(data_rot))
    if idx[0].shape[0]>0:
        print (data_rot)
    
        print ("data: ", data)
        return None
    
    #print ("data rot: ", data_rot.shape)
    # return rotated body, angle and translation pt
    return data_rot #, angle #, translation_pt



#    
def get_vectors(animal_id, vectors_idx, feature_ids, min_duration):

    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    d = np.load(root_dir + '/animalID_'+
                str(animal_id)+
                '_alldata_imputed.npy')
    
    print (d.shape)
    
    # 
    vecs = []
    for k in trange(vectors_idx.shape[0]):
        temp = d[vectors_idx[k][0]:vectors_idx[k][1]][:,feature_ids]
        idx = np.where(np.isnan(temp))
        
        if idx[0].shape[0]>0:
            print (k, vectors_idx[k][0], vectors_idx[k][1])
            print (temp)
            break
            
        vecs.append(temp)
        
    vecs = np.array(vecs)
    
    return vecs


# 
def vectors_to_egocentric(vecs, animal_id,min_duration):
    
    # print ("vecs: ", vecs.shape)

    fname_out = '/home/cat/vecs_ego_animalID_'+str(animal_id)+"_duration_"+str(min_duration)+'.npy'
    if os.path.exists(fname_out)==False:
        vecs_ego = np.zeros(vecs.shape, 'float32')+np.nan

        # 
        for s in trange(vecs.shape[0],  desc='Getting egocentric vectors', leave=True):

            vec = vecs[s]
            #print ("vec: ", vec.shape)
            # centre and align data
            vecs_ego[s] = centre_and_align2_pairwise(vec)

        np.save(fname_out, vecs_ego)
    else:
        vecs_ego = np.load(fname_out)
            
    return vecs_ego

def find_movements(vecs_ego):
    
    print ("vecs: ", vecs_ego.shape)


    min_quiet_n_frames = 2      # number of frames of static
    min_quiet_movement = 1
    
    #
    min_velocity = 1             # minimum pixels to move to indicate movement intiaitins 
    
    
    # 
    ctr_q = 0
    idx_movement = []
    s = 0
    while s<vecs_ego.shape[0]:
#    for s in trange(vecs_ego.shape[0]):
        
        # check if there is at least 10 frames quiet followed by high movement
        
        # get location of noses
        noses = vecs_ego[s,:,0]
        #print ("noses: ", noses.shape)
        
        # get velocities
        temp = noses[1:]-noses[:-1]
        vel = np.linalg.norm(temp,axis=1)
        #print ("vel: ", vel, vel.shape)
        
        # find periods with at least 10 frames of low or no movement
        if np.max(vel[:min_quiet_n_frames])<=min_quiet_movement:
            ctr_q+=1
            
            # require minimum movement in next frame
            if True:
                if vel[min_quiet_n_frames]>=min_velocity:
                    idx_movement.append(s)
                    #vecs_movement.append(vecs_ego[s])
            else:
                #vecs_movement.append(vecs_ego[s])
                idx_movement.apppend(s)
        s+=1
        
        
    print ("ctrq: ", ctr_q)
    idx_movement = np.array(idx_movement)
        
    return idx_movement


def dim_red(X_pca):
    from sklearn import decomposition
    import sklearn

    # 
    print ("X_pca: ", X_pca.shape)

    #
    if False:
        pca = decomposition.PCA(n_components=3)

        X_pca = pca.fit_transform(X_pca)
        print (X_pca.shape)

    else:
        import umap
        umap = umap.UMAP(n_components=2,
                        init='random',
                        random_state=0)

        print ("fitting umap")
        #umap_ = umap.fit(vecs_pca[::10])
        umap_ = umap.fit(X_pca)

        print ("transforming alldata")
        X_pca = umap_.transform(X_pca)


    print ("plotting: ", X_pca.shape)
    
    return X_pca
    

# 
def make_pca_data(vecs_movement, angles):
    
    vecs_nose = vecs_movement[:,:,0]
    print ("vecs_nose: ", vecs_nose.shape)
    
    X_pca = np.zeros((vecs_movement.shape[0], vecs_movement.shape[1],vecs_movement.shape[2]+1))
    
    for k in range(X_pca.shape[0]):
        X_pca[k,:,:2] = vecs_nose[k]
        X_pca[k,:,2] = angles[k]
        
    
    print (X_pca.shape)
    
    X_pca = X_pca.reshape(X_pca.shape[0],-1)
    
    return X_pca 

from numpy import arccos, array
from numpy.linalg import norm

# Note: returns angle in radians
def theta(v, w): 
    return arccos(v.dot(w)/(norm(v)*norm(w)))


def get_angles(vecs, animal_id, min_duration):

    fname_out = '/home/cat/angles_ego_animalID_'+str(animal_id)+"_duration_"+str(min_duration)+'.npy'
     
    if os.path.exists(fname_out)==False:
        angles = np.zeros((vecs.shape[0], 
                           vecs.shape[1]),
                           'float32')+np.nan
        # 
        for f in trange(vecs.shape[0],  desc='Getting angles', leave=True):

            # 
            temp1 = vecs[f,0]
            temp1 = temp1[1] - temp1[0]
            angle1 = np.angle(complex(*(temp1)))
            for m in range(0,vecs.shape[1],1):

                # 
                temp2 = vecs[f,m]
                temp2 = temp2[1] - temp2[0]
                angle2 = np.angle(complex(*(temp2)), deg=True)

                angle = angle1-angle2
                angles[f,m]=angle
        
        np.save(fname_out, angles)
    else:
        angles = np.load(fname_out)
        
    return angles

def get_angles2(vecs, vecs_ego, animal_id, min_duration):
    import math
    
    fname_out = '/home/cat/angles_ego_animalID_'+str(animal_id)+"_duration_"+str(min_duration)+'.npy'
     
    if os.path.exists(fname_out)==False:
        angles = np.zeros((vecs.shape[0], 
                           vecs.shape[1]),
                           'float32')+np.nan
        # 
        for f in trange(vecs.shape[0],  desc='Getting angles', leave=True):

            # 
            temp1 = vecs[f,0]
            temp1 = temp1[1] - temp1[0]
            angle1 = np.angle(complex(*(temp1)))
            for m in range(0,vecs.shape[1],1):

                # 
                # 
                temp2 = vecs[f,m]
                temp2 = temp2[1] - temp2[0]
                
                angle = math.atan2(temp1[0]*temp2[1] - temp1[1]*temp2[0], 
                              temp1[0]*temp2[0] + temp1[1]*temp2[1])

                angles[f,m]=angle
        
        np.save(fname_out, angles)
    else:
        angles = np.load(fname_out)
        

    return angles
# 


def get_acceleration(vecs, animal_id, min_duration):
    
    fname_out = '/home/cat/acceleration_ego_animalID_'+str(animal_id)+"_duration_"+str(min_duration)+'.npz'
    
    acc_ap = np.zeros((vecs.shape[0],vecs.shape[1]-2),'float32')
    acc_ml = np.zeros((vecs.shape[0],vecs.shape[1]-2),'float32')
    
    if os.path.exists(fname_out)==False:
        for f in trange(vecs.shape[0], desc='Getting acceleration'):

            vecs_nose = vecs[f,:,0]
            #print ("vecs_nose: ", vecs_nose.shape)

            vel_ap = vecs_nose[1:,0] - vecs_nose[:-1,0]
            vel_ml = vecs_nose[1:,1] - vecs_nose[:-1,1]

            #
            aa_ap = vel_ap[1:]-vel_ap[:-1]
            aa_ml = vel_ml[1:]-vel_ml[:-1]
            
            #print ("aa_ap: ", aa_ap.shape)
            acc_ap[f]=aa_ap
            acc_ml[f]=aa_ml

        np.savez(fname_out,
                acc_ap=acc_ap,
                acc_ml=acc_ml)
    else:
        d = np.load(fname_out)
        acc_ap = d['acc_ap']
        acc_ml = d['acc_ml']
   
    return acc_ap, acc_ml



def generate_egocentric_and_angles_data(animal_ids, min_duration):

    vecs_mov_array = []
    # 
    for animal_id in animal_ids:

        #
        ##################################
        ##################################
        ##################################
        two, six, headnose = get_lengths(animal_id)

        #
        ##################################
        ##################################
        ##################################
        _,_,vectors_idx = get_durations(headnose, 0, 
                                        animal_id, 
                                        min_duration = min_duration, 
                                        plotting = False)
                
        ##################################
        ##################################
        ##################################
        feature_ids = np.arange(2)
        vecs = get_vectors(animal_id, vectors_idx, feature_ids, min_duration)

        ##################################
        ##################################
        ##################################
        vecs_ego = vectors_to_egocentric(vecs, animal_id, min_duration)

        ##################################
        ##################################
        ##################################
        angles = get_angles2(vecs, vecs_ego, animal_id, min_duration)

        ##################################
        ##################################
        ##################################
        _,_ = get_acceleration(vecs_ego,  animal_id, min_duration)


        print ("angles: ", angles.shape)

        print ('')

def plot_angle_acceleration_distributions(animal_ids, min_duration):
    fps = 25
    # 
    vecs_mov_array = []
    for animal_id in animal_ids:
        
        ##################################
        ##################################
        ##################################
        vecs = None                        # if this is already computed it is not required
        vecs_ego = vectors_to_egocentric(vecs, animal_id, min_duration)

        ##################################
        ##################################
        ##################################
        angles = get_angles2(vecs, vecs_ego, animal_id, min_duration)

        ##################################
        ##################################
        ##################################
        acc_ap, acc_ml = get_acceleration(vecs_ego,  animal_id, min_duration)

        ##################################
        ######### PLOT ANGLES ############
        ##################################
        plt.subplot(4,4,animal_id*4+1)

        print ("angles: ", angles.shape)
        print ('acc: ', acc_ap.shape)
        width = 1
        rad_to_degree= 57.2958
        lims = 360
        bins = np.arange(-lims,lims+width, width)
        temp = angles.flatten()*fps*rad_to_degree


        y = np.histogram(temp, bins=bins)
        plt.plot(y[1][1:]-width/2, y[0],c='black')
        plt.semilogy()
        plt.ylim(bottom=1)
        plt.plot([0,0],[0,np.max(y[0])],'--')
        plt.plot([45,45],[0,np.max(y[0])],'--')
        plt.plot([-45,-45],[0,np.max(y[0])],'--')
        if animal_id==0:
            plt.title("angles (deg/sec) pdf")
        plt.ylabel("Animal "+str(animal_id))

        ##################################
        ######### PLOT ACC_AP ############
        ##################################
        plt.subplot(4,4,animal_id*4+2)
        lims = 0.5*fps
        width = .01*fps
        bins = np.arange(-lims,lims+width, width)
        temp = acc_ap.flatten()
        y = np.histogram(temp, bins=bins)
        plt.plot(y[1][1:]-width/2., y[0],c='blue')
        plt.semilogy()
        plt.ylim(bottom=1)
        plt.plot([0,0],[0,np.max(y[0])],'--')
        if animal_id==0:
            plt.title("acceleration ap (pix/sec) pdf")

        ##################################
        ######### PLOT ACC_ML ############
        ##################################
        plt.subplot(4,4,animal_id*4+3)
        #width = 1
        bins = np.arange(-lims,lims+width, width)
        temp = acc_ml.flatten()
        y = np.histogram(temp, bins=bins)
        plt.plot(y[1][1:]-width/2, y[0],c='red')    
        plt.semilogy()
        plt.plot([0,0],[0,np.max(y[0])],'--')
        plt.ylim(bottom=1)
        if animal_id==0:
            plt.title("acceleration ml (pix/sec) pdf")

        ##################################
        ###### PLOT ACC-OVERALL ##########
        ##################################
        plt.subplot(4,4,animal_id*4+4)
        width = 5
        lims = 500
        bins = np.arange(0,lims+width, width)
        temp1 = acc_ml.flatten()
        temp2 = acc_ap.flatten()
        temp3 = np.sqrt(acc_ml**2+acc_ap**2)*25

        #
        y = np.histogram(temp3, bins=bins)
        plt.plot(y[1][1:]-width/2, y[0],c='magenta')    
        plt.semilogy()
        #plt.semilogx()
        plt.plot([0,0],[0,np.max(y[0])],'--')
        plt.plot([35,35],[0,np.max(y[0])],'--')
        plt.plot([150,150],[0,np.max(y[0])],'--')
        plt.plot([245,245],[0,np.max(y[0])],'--')
        plt.ylim(bottom=1)
        if animal_id==0:
            plt.title("abs acceleration (pix/sec) pdf")   

        print ('')
        

def compute_discretized_angles_and_acceleration(animal_id, min_duration):
    fps = 25
    rad_to_degree= 57.2958

    # discretized thresholds for angles 
    angles_thresh = [[-1E8, -45],
                     [-45,+45],
                     [+45,1E8]
                    ]
    
    # discretized thresholds for acceleration 
#     acc_thresh = [[0,30],
#                   [30,75],
#                   [75,150],
#                   [150,1E8]]
    
#     acc_thresh = [[0,30],
#                   [30,1E8]]

    acc_thresh = [[0,30],
                  [30,75],
                  [75,1E8]]
        
    #
    
    #
    print ("agnels thresholds: ", angles_thresh)
    
    # 
    print ("accelaration thresholds: ", acc_thresh)
    

    ##################################
    ##################################
    ##################################
    vecs = None                        # if this is already computed it is not required
    vecs_ego = vectors_to_egocentric(vecs, animal_id, min_duration)

    ##################################
    ##################################
    ##################################
    angles = get_angles2(vecs, vecs_ego, animal_id, min_duration)
    #print ("ANGLES: ", angles)

    ##################################
    ##################################
    ##################################
    acc_ap, acc_ml = get_acceleration(vecs_ego, animal_id, min_duration)
    
    ##################################
    ###### SAVE CONTINOUS DATA #######
    ##################################    
    # process acceleration
    temp1 = acc_ml.flatten()
    temp2 = acc_ap.flatten()
    acc = np.sqrt(acc_ml**2+acc_ap**2)*25
    
    print ("angles and everything else: ", angles.shape, acc_ap.shape, acc_ml.shape)
    all_continuous = np.hstack((angles[:,2:], acc_ap, acc_ml))
    print ("all_continuous: ", all_continuous.shape)
    # 
    np.save('/home/cat/all_continuous_'+str(animal_id)+
            '_min_duration'+str(min_duration)+
            '.npy', all_continuous)    
    
    ##################################
    ######### DISCRETIZE #############
    ##################################
    angles_discretized = np.zeros(angles.shape, 'float32')+np.nan
    for k in trange(angles.shape[0]):
        temp = angles[k]*fps*rad_to_degree
        
        for a in range(len(angles_thresh)):
            idx = np.where(np.logical_and(
                                temp>=angles_thresh[a][0],
                                temp<angles_thresh[a][1],
                           ))[0]
            #temp[idx]=a
            
            angles_discretized[k,idx]= a
        
        
    # discretize accelaration
    acc_discretized = np.zeros(acc.shape, 'float32')+np.nan
    for k in trange(acc.shape[0]):
        temp = acc[k]

        for a in range(len(acc_thresh)):
            idx = np.where(np.logical_and(
                                temp>=acc_thresh[a][0],
                                temp<acc_thresh[a][1],
                           ))[0]

            acc_discretized[k,idx]= a

            
    # 
    all_discretized = np.hstack((angles_discretized[:,2:], acc_discretized))
    
    # 
    np.save('/home/cat/all_discretized_'+str(animal_id)+
            '_min_duration'+str(min_duration)+
            '.npy', all_discretized)
    
    
    return (all_continuous,
            all_discretized,
            angles_discretized, 
            acc_discretized, 
            angles,
            acc,
            angles_thresh, 
            acc_thresh)

# # 
# def dim_red2(animal_id,
#              min_duration,
#              angles_thresh,
#              acc_thresh,
#              method,
#              subsample=5):
    
#     #
#     from sklearn import decomposition
#     import sklearn
    
#     #
#     root_dir = '/media/cat/1TB/dan/cohort1/slp/'

#     fname_out = os.path.join(root_dir,"DimRed_animalID_"+str(animal_id)+
#                              "_min_duration_"+str(min_duration)+
#                              "_method_"+str(method)+
#                              "_n_events_"+str(len(angles_thresh))+
#                              '.npz'
#                             )
    
#     #
#     if os.path.exists(fname_out)==False:

#         X_pca = np.zeros((angles_thresh.shape[0], 
#                       acc_thresh.shape[1], 2), 'float32')
#         X_pca[:,:,0]=angles_thresh[:,2:]
#         X_pca[:,:,1]=acc_thresh
#         print ("Xpca: ", X_pca.shape)

#         #
#         X_pca = X_pca.reshape(X_pca.shape[0],-1)
#         print ("Xpca: ", X_pca.shape)


#         ###################################
#         ###################################
#         ###################################
#         if method==0:
#             pca = decomposition.PCA(n_components=3)

#             X_pca_fit = pca.fit_transform(X_pca)
#             print (X_pca_fit.shape)

#         elif method==1:
#             import umap
#             umap = umap.UMAP(n_components=2,
#                             init='random',
#                             random_state=0)

#             X_pca_subsampled = X_pca[::subsample]
#             print ("fitting umap on" ,X_pca_subsampled.shape)
#             umap_ = umap.fit(X_pca_subsampled)

#             print ("transforming alldata ", X_pca.shape)
#             X_pca_fit = umap_.transform(X_pca)

#         # retransform X_pca to regular shape
#         X_pca = X_pca.reshape(X_pca.shape[0], -1, 2)
    
#         # 
#         np.savez(fname_out,
#                  X_pca_fit = X_pca_fit,
#                  X_pca = X_pca,
#                  animal_id = animal_id,
#                  min_duration = min_duration,
#                  method = method,
#                  subsample = subsample,
#                 )
#     else:
#         data = np.load(fname_out,allow_pickle=True)
#         X_pca_fit = data["X_pca_fit"]
#         X_pca = data['X_pca']
    
    
#     return X_pca_fit, X_pca



# CLUSTER STEP
def cluster_and_visualize(X_pca_fit, 
                          X_pca, 
                          n_clusters=2, 
                          cluster_id = None):
    
    #clrs = ['black','blue','red','green', 'magenta','brown','cyan','yellow',
    #        'pink','olive']
       
    import matplotlib.cm as cm
    from matplotlib.colors import Normalize

        
    # use GaussianMixture clusters
    if n_clusters is not None:
        from sklearn.mixture import GaussianMixture

        gmm = GaussianMixture(n_components=n_clusters).fit(X_pca_fit)
        labels = gmm.predict(X_pca_fit)
        print(np.unique(labels))
          # PLOT STEP
        n_clusters = np.unique(labels).shape[0]
        print ("n_clusters: ", n_clusters, np.unique(labels))
        colors = plt.cm.tab20(np.linspace(0,1,20))

    # use metadata
    else:
        labels = []
#         for k in trange(X_pca.shape[0], desc='Making colors'):
#             #print (X_pca[k].shape)
#             temp = X_pca[k]
#             ave_angle = np.median(temp[:,0])
#             ave_acc = np.median(temp[:,1])
            
#             labels.append(ave_angle+ave_acc*3)
    
            
#         labels = np.int32(labels)
#         print ("labels: ", labels)
  
        #  
        if True:
            medians = np.mean(X_pca, axis=1)
            print ("means: ", medians.shape)
            
            labels = np.log(medians[:,0]+medians[:,1]*3)
            print ("Labels log: ", labels)
                        
            labels = (labels-np.min(labels))/(np.max(labels)-np.min(labels))
            print ("Labels norm: ", labels)
            
            labels = np.abs(np.log(labels+0.000001))
            print ("labels: ", labels)
            
            labels = (labels-np.min(labels))/(np.max(labels)-np.min(labels))
            print ("labels: ", labels)
            
            labels = np.around(labels, 1)
            labels = 1-labels
            print ("Final labels: ", labels)
            
            cmap = cm.viridis

            colors = plt.cm.viridis(np.linspace(0,1,100))

            ctr=0
            for k in np.unique(labels):
                plt.subplot(2,3,ctr+1)

                idx = np.where(labels==k)[0]
                print ("k: ", k, idx.shape)
                plt.scatter(X_pca_fit[idx,0],
                            X_pca_fit[idx,1],
                            alpha=.1,
                            edgecolor='black',
                            #color = colors[labels]
                            color = cmap(labels[idx])
                            #color = colors[labels[idx]]
                       )
                ctr+=1
                plt.xlim(-15,25)
                plt.ylim(-20,30)


            return idx, labels
        
        
        else: 
            medians = np.median(X_pca, axis=1)
            labels = np.log(medians[:,0]+medians[:,1]*3)
            print ("labels: ", labels, np.unique(labels))
            labels = (labels-np.min(labels))/(np.max(labels)-np.min(labels))
            print ("labels: ", labels, np.unique(labels))
            labels = np.int32(labels*99)

            cmap = cm.tab20c
            colors = plt.cm.viridis(np.linspace(0,1,100))
            #norm = Normalize(vmin=np.min(labels), vmax=np.max(labels))
            
            fig=plt.figure()
            ctr=0
            for k in np.unique(labels):
                plt.subplot(2,3,ctr+1)

                idx = np.where(labels==k)[0]
                plt.scatter(X_pca_fit[idx,0],
                        X_pca_fit[idx,1],
                        alpha=.1,
                        edgecolor='black',
                        #color = colors[labels]
                        #color = cmap(labels)
                        color = colors[labels[idx]]
                       )
                ctr+=1
                plt.xlim(-15,25)
                plt.ylim(-20,30)
            
            return idx, labels

    # 
    if cluster_id is None:
        fig=plt.figure()

        for id_ in np.unique(labels):
            idx = np.where(labels==id_)[0]
            print ("cluster: ", id_, " has # events: ", idx.shape[0])

            plt.scatter(X_pca_fit[idx,0],
                        X_pca_fit[idx,1],
                        alpha=.1,
                        edgecolor='black',
                        color = colors[labels[idx]]
                       )

    # 
    print ("PLOTTING...")
    plt.show()
    
    return idx, labels


               
# 


def get_normalized_histograms(angles_discretized,
                              acc_discretized):
    
    print ("COMPUTING Klaus normalized histograms")
    #
    bins = np.arange(np.unique(angles_discretized).shape[0]+1)-0.5
    print ("BINS angles: ", bins)
    angles_hist = np.zeros((angles_discretized.shape[0], 
                             bins.shape[0]-1), 'float32')
    for k in trange(angles_discretized.shape[0]):
        angles_hist[k] = np.histogram(angles_discretized[k], bins = bins)[0]
   
    # normalize
    angles_hist = angles_hist / np.sum(angles_hist[0])

    #
    bins = np.arange(np.unique(acc_discretized).shape[0]+1)-0.5
    print ("BINS accel: ", bins)
    acc_hist = np.zeros((acc_discretized.shape[0], 
                             bins.shape[0]-1), 'float32')
    for k in trange(acc_discretized.shape[0]):
        acc_hist[k] = np.histogram(acc_discretized[k], bins = bins)[0]
      
    # normalize
    acc_hist = acc_hist / np.sum(acc_hist[0])
    
    #
    print ("Angles histograms: ", angles_hist.shape)
    print ("Accel histograms: ", acc_hist.shape)
    print ("")
    
    return  angles_hist, acc_hist


def get_unique_angles_accs(angles_hist, acc_hist):
    # 
    ang_unique = np.unique(angles_hist, axis=0)
    print ("Unique angles: ", ang_unique.shape)

    acc_hist = np.around(acc_hist, 3)
    acc_unique = np.unique(acc_hist, axis=0)
    print ("Unique accel: ", acc_unique.shape)

    # 
    all_hist = np.hstack((angles_hist, acc_hist))
    all_unique = np.unique(all_hist, axis=0)

    print ("# possible combinations: ", acc_unique.shape[0]*ang_unique.shape[0])
    print ("# of actual combinations: ", all_unique.shape)
    
    
    print ("normalized angles histograms: ", angles_hist[50:52])
    print ("ang unique: ", ang_unique[:2])
    print ("acc unique: ", acc_unique[:2])

    return ang_unique, acc_unique, all_unique


    
    


def compute_emd_normalized_unique_angle_acc(ang_unique, acc_unique):
    emd_angle=np.zeros((ang_unique.shape[0],ang_unique.shape[0]))
    for i in trange(ang_unique.shape[0]):
        for p in range(i+1, ang_unique.shape[0],1):
            emd_angle[i,p] = scipy.stats.wasserstein_distance(ang_unique[i], 
                                                              ang_unique[p])
            
    # normalize by n-1
    emd_angle = emd_angle / ang_unique.shape[1]

    #
    emd_acc=np.zeros((acc_unique.shape[0],acc_unique.shape[0]))
    for i in trange(acc_unique.shape[0]):
        for p in range(i+1, acc_unique.shape[0],1):
            emd_acc[i,p] = scipy.stats.wasserstein_distance(acc_unique[i], 
                                                            acc_unique[p])
            
    # normalize by n-1
    emd_acc = emd_acc / acc_unique.shape[1]
    
    
    return emd_angle, emd_acc


def get_index(array, feature_array):
    
    idx = np.sum(np.equal(feature_array, array), axis=1)
    idx = np.where(idx==len(array))[0]

    return idx


def compute_S_matrix(indexes, all_unique, ang_unique, acc_unique, n_split):

    root_dir = '/media/cat/1TB/dan/cohort1/slp/'

    fname_out = os.path.join(root_dir,"S_unique","S_unique_index"+str(indexes[0])+"_nsplit_"+str(n_split)+'.npz')
    
    if os.path.exists(fname_out)==False:

        S_ang = np.zeros((len(indexes),all_unique.shape[0]),'float32') #+np.nan
        S_acc = np.zeros((len(indexes),all_unique.shape[0]),'float32') #+np.nan

        #for p in trange(all_unique.shape[0]):
        for ctrp, p in enumerate(indexes):

            # find outer loop all feature match to ang unique features
            idx1_ang = get_index(all_unique[p,:3], 
                                   ang_unique)
            # same for acceleration
            idx1_acc = get_index(all_unique[p,3:], 
                                   acc_unique)

            # loop over inner dimensions
            for q in range(p+1, all_unique.shape[0],1):

                # get index of current frame for angle
                idx2_ang = get_index(all_unique[q,:3], 
                                     ang_unique)
                # 
                S_ang[ctrp,q] = emd_angle_unique_norm[idx1_ang,idx2_ang] 

                # get index for acceleration
                idx2_acc = get_index(all_unique[q,3:], 
                                     acc_unique)

               # print (idx1_ang,idx2_ang,idx1_acc,idx2_acc )
                # 
                S_acc[ctrp,q] = emd_acc_unique_norm[idx1_acc,idx2_acc] 

        # compute sum and normalize

        S = -((S_ang + S_acc)/2)**2
        #print (indexes[0], " S: ", S.shape)

        np.savez(fname_out,
                S = S,
                indexes = indexes)
         
    



In [3]:
######################################
######## PREPROCESS DATA #############
######################################
# 
animal_ids = [0]
min_duration = 15
generate_egocentric_and_angles_data(animal_ids, min_duration)
    

animal:  0 all data imputed:  (2069710, 6, 2)


  8%|▊         | 24579/306873 [00:00<00:02, 123185.90it/s]

(2069710, 6, 2)


100%|██████████| 306873/306873 [00:02<00:00, 125930.12it/s]


angles:  (306873, 15)



In [100]:
##################################
######### PLOT DISTRIBUTIONS #####
##################################
animal_ids = [0]
#min_duration = 50
plot_angle_acceleration_distributions(animal_ids, min_duration)


angles:  (306873, 15)
acc:  (306873, 13)



In [4]:
##################################
######### GET PCA DATA ###########
##################################
animal_id = 0
#min_duration = 25
(all_continuous,
all_discretized,
angles_discretized, 
acc_discretized, 
angles,
acc,
angles_thresh, 
acc_thresh) =  compute_discretized_angles_and_acceleration(animal_id, 
                                                           min_duration)

# 


  2%|▏         | 5056/306873 [00:00<00:05, 50548.60it/s]

agnels thresholds:  [[-100000000.0, -45], [-45, 45], [45, 100000000.0]]
accelaration thresholds:  [[0, 30], [30, 75], [75, 100000000.0]]
angles and everything else:  (306873, 15) (306873, 13) (306873, 13)
all_continuous:  (306873, 39)


100%|██████████| 306873/306873 [00:04<00:00, 63926.63it/s]


In [173]:
######################################
######## COMPUTE HISTOGRAMS ##########
######################################
#
angles_hist, acc_hist = get_normalized_histograms(angles_discretized,
                                                  acc_discretized)


def get_random_sample_angles_accs(angles_hist, acc_hist, subsample):

    # round acc and ang to 3 decimals; Doesn't make a real difference in the end;
    angles_hist = np.around(angles_hist, 3)
    acc_hist = np.around(acc_hist, 3)
        
    # uniformly subsample the data:
    idx = np.arange(angles_hist.shape[0])[::subsample]
    
    
    all_histograms = np.hstack((angles_hist, acc_hist))[idx]
    print (idx.shape, angles_hist.shape, "all_histograms: ", all_histograms.shape)
    
    ang_unique = np.unique(all_histograms[:,:3], axis=0)
    acc_unique = np.unique(all_histograms[:,3:], axis=0)
    
    print ("Ang unique: ", ang_unique.shape, "acc unique: ", acc_unique.shape)   
    
    return ang_unique, acc_unique, all_histograms

#
subsample = 1
ang_unique, acc_unique, all_histograms = get_random_sample_angles_accs(angles_hist, 
                                                                       acc_hist,
                                                                       subsample = subsample)

np.save('/home/cat/all_histograms_min_duration'+str(min_duration)+'.npy', all_histograms)
#

  2%|▏         | 4021/216702 [00:00<00:05, 40201.32it/s]

COMPUTING Klaus normalized histograms
BINS angles:  [-0.5  0.5  1.5  2.5]


  2%|▏         | 3914/216702 [00:00<00:05, 39136.49it/s]

BINS accel:  [-0.5  0.5  1.5  2.5]


100%|██████████| 216702/216702 [00:05<00:00, 40866.53it/s]


Angles histograms:  (216702, 3)
Accel histograms:  (216702, 3)

(216702,) (216702, 3) all_histograms:  (216702, 6)
Ang unique:  (36, 3) acc unique:  (28, 3)


In [10]:
from sklearn.manifold import TSNE
from sklearn import decomposition
import sklearn

def dim_red3(X,
             method,
             subsample_fit=5,
             subsample_predict=1,
             n_components=2):
    
    ###################################
    ###################################
    ###################################
    if method==0:
        pca = decomposition.PCA(n_components=n_components)

        X_fit = pca.fit_transform(X)
        print (X_fit.shape)

    elif method==1:
        import umap
        umap = umap.UMAP(n_components=n_components,
                         #n_neighbors=100,
                         init='random',
                         random_state=0)

        X_subsampled = X[::subsample_fit]
        print ("fitting umap on" ,X_subsampled.shape)
        umap_ = umap.fit(X_subsampled)

        print ("transforming alldata ", X.shape)
        X_fit = umap_.transform(X[::subsample_predict])
   
    elif method==2:
        #X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
        X_subsampled = X[::subsample_fit]
        print ("fitting tsne on" ,X_subsampled.shape)
        tsne_ = TSNE(n_components=2).fit(X)
        X_fit = tsne_.transform(X[::subsample_predict])
    
    elif method==3:
        
        # use principled subsampling to select at least 1 of each unique vector        
        X_unique, indices, counts = np.unique(X, axis=0, 
                                              return_counts=True, 
                                              return_index=True)
#         print ("X_unique: ", X_unique.shape, X_unique[0])
#         print ("counts: ", counts.shape, counts[0])
#         print ("indices: ", indices.shape, indices[0])
        
        X_sampled = []
        for k in range(X_unique.shape[0]):
            ct = max(1, int(counts[k]/subsample_fit))
            #print (k, ct, counts[k])
            for c in range(ct):
                X_sampled.append(X_unique[k])

        X_sampled = np.array(X_sampled)
        print ("X_sampled (min 1 instance of each unique vector): ", X_sampled.shape)
        
        import umap
        umap = umap.UMAP(n_components=n_components,
                         #n_neighbors=100,
                         init='random',
                         random_state=0)
        # 
        X_subsampled = X_sampled
        print ("fitting umap on" ,X_subsampled.shape)
        umap_ = umap.fit(X_subsampled)
        
        # 
        print ("transforming alldata ", X.shape)
        X_fit = umap_.transform(X[::subsample_predict])
        
    return X_fit


# ########################################
# ########################################
# ########################################
data_type = 'continuous'
if data_type=='histograms':
    X = all_histograms
elif data_type == 'discretized':
    X = all_discretized
elif data_type == 'continuous':
    X = all_continuous

    
fname_out = ('/home/cat/umap_x_fit_min_duration_'+
        str(min_duration)+
        '_dataType_'+data_type+'.npy')

if os.path.exists(fname_out)==False:
    #labels = clustering.labels_
    X_fit = dim_red3(X,
                     method=1,
                     subsample_fit=10,
                     subsample_predict=1,
                     n_components = 2)

    # # 

    np.save(fname_out, 
            X_fit)
    
else:
    X_fit = np.load(fname_out)

########################################
########################################
########################################
print ("X_fit: ", X_fit.shape)

fig = plt.figure(figsize=(15,15))
ax=plt.subplot(111)
plt.scatter(X_fit[:,0], X_fit[:,1],
           # edgecolor='black',
            s=1,
            alpha=.01)
plt.title("Min_duration "+str(min_duration)+
          " data type: "+ data_type)

if True:
    plt.savefig(fname_out.replace('.npy','.png'),dpi=300)
    plt.close()

else:
    plt.show()


X_fit:  (306873, 2)


In [11]:
##################################################
########## SEARCH UMAP PARAMETER SPACE ###########
##################################################

# 
def run_umap_params(X, 
                    min_duration,
                    n_neighbors, 
                    min_dist, 
                    subsample_fit = 10,
                    subsample_predict = 1,  # do not subsample at prediction time
                    n_components = 2
                  ):
    # 
    fname_out = ('/home/cat/umap_x_fit_min_duration_'+str(min_duration)+
                  '_dataType_'+data_type+
                  '_minDist_'+str(min_dist)+
                  '_nNeighbors_'+str(n_neighbors)+
                  '.npy')

    # 
    if os.path.exists(fname_out)==False:

        X_unique, indices, counts = np.unique(X, axis=0, 
                                              return_counts=True, 
                                              return_index=True)
        # 
        X_sampled = []
        for k in range(X_unique.shape[0]):
            ct = max(1, int(counts[k]/subsample_fit))
            #print (k, ct, counts[k])
            for c in range(ct):
                X_sampled.append(X_unique[k])

        X_sampled = np.array(X_sampled)
        print ("X_sampled (min 1 instance of each unique vector): ", X_sampled.shape)

        import umap
        umap = umap.UMAP(n_components=n_components,
                         n_neighbors=n_neighbors,
                         min_dist = min_dist,
                         init='random',
                         random_state=0)
        # 
        
        if data_type == 'continuous':
            print ("fitting / tranforming all", X.shape)
            X_fit = umap.fit_transform(X)

            
        else:
            X_subsampled = X_sampled
            print ("fitting umap on" ,X_subsampled.shape)
            umap_ = umap.fit(X_subsampled)

            # 
            print ("transforming alldata ", X.shape)
            X_fit = umap_.transform(X[::subsample_predict])


        # 
        np.save(fname_out, 
                X_fit)

        ########################################
        ########################################
        ########################################
        print ("X_fit: ", X_fit.shape)

#         fig = plt.figure(figsize=(15,15))
#         ax=plt.subplot(111)
#         plt.scatter(X_fit[:,0], X_fit[:,1],
#                    # edgecolor='black',
#                     s=1,
#                     alpha=.01)
#         plt.title(fname_out,fontsize=8)

#         if True:
#             plt.savefig(fname_out.replace('.npy','.png'),dpi=300)
#             plt.close()

    else:
        X_fit = np.load(fname_out)

    ########################################
    ########################################
    ########################################
#     print ("X_fit: ", X_fit.shape)

#     fig = plt.figure(figsize=(15,15))
#     ax=plt.subplot(111)
#     plt.scatter(X_fit[:,0], X_fit[:,1],
#                # edgecolor='black',
#                 s=1,
#                 alpha=.01)
#     plt.title(fname_out,fontsize=8)

#     if True:
#         plt.savefig(fname_out.replace('.npy','.png'),dpi=300)
#         plt.close()


    return X_fit

###############################################
###############################################
###############################################
data_type = 'continuous'
if data_type=='histograms':
    X = all_histograms
elif data_type == 'discretized':
    X = all_discretized
elif data_type == 'continuous':
    X = all_continuous

    #
ctr=1
matplotlib.use('Agg')
fig=plt.figure(figsize=(15,15))
for min_dist in (0.0, 0.1, 0.25, 0.5, 0.8, 0.99):
    for n_neighbors in (2, 5, 10, 20, 50, 100, 200):
        
        X_fit = run_umap_params(X, 
                    min_duration,
                    n_neighbors, 
                    min_dist,
                    subsample_fit = 10,
                    subsample_predict = 1,  # do not subsample at prediction time
                    n_components = 2)
        
        plt.subplot(6,7,ctr)
        plt.scatter(X_fit[:,0], X_fit[:,1],
               # edgecolor='black',
                s=0.1,
                alpha=.01)
        
        plt.xticks([])
        plt.yticks([])
        ctr+=1
        
if True:
    plt.savefig("/home/cat/xfit.png",dpi=600)
    plt.close()
    
else:
    plt.show()

In [110]:
# from sklearn.mixture import GaussianMixture

# ##############################
# ##############################
# ##############################
# n_clusters = 40
# gmm = GaussianMixture(n_components=n_clusters).fit(X_fit)
# labels = gmm.predict(X_fit)
# print(labels)

# ##############################
# ##############################
# ##############################
# n_clusters = np.unique(labels).shape[0]
# print ("n_clusters: ", n_clusters, np.unique(labels))
# colors = plt.cm.viridis(np.linspace(0,1,n_clusters))

# ##############################
# ##############################
# ##############################
# for id_ in np.unique(labels):
#     idx = np.where(labels==id_)[0]
#     print ("cluster: ", id_, " has # events: ", idx.shape[0])

#     plt.scatter(X_fit[idx,0],
#                 X_fit[idx,1],
#                 alpha=.1,
#                 edgecolor='black',
#                 color = colors[labels[idx]]
#                )

# plt.show()
    
    
#   # PLOT STEP
# n_clusters = np.unique(labels).shape[0]
# print ("n_clusters: ", n_clusters, np.unique(labels))
# colors = plt.cm.tab20(np.linspace(0,1,20))

        

[30 20 13 ... 11 13 11]
n_clusters:  40 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
cluster:  0  has # events:  24825
cluster:  1  has # events:  904
cluster:  2  has # events:  3983
cluster:  3  has # events:  2169
cluster:  4  has # events:  3449
cluster:  5  has # events:  10095
cluster:  6  has # events:  35199
cluster:  7  has # events:  2079
cluster:  8  has # events:  3491
cluster:  9  has # events:  3600
cluster:  10  has # events:  3246
cluster:  11  has # events:  2481
cluster:  12  has # events:  1864
cluster:  13  has # events:  27436
cluster:  14  has # events:  3487
cluster:  15  has # events:  2736
cluster:  16  has # events:  4940
cluster:  17  has # events:  3532
cluster:  18  has # events:  1428
cluster:  19  has # events:  24351
cluster:  20  has # events:  4089
cluster:  21  has # events:  3308
cluster:  22  has # events:  23752
cluster:  23  has # events:  4609
cluster:  24  has # events

In [84]:
#######################################
#######################################
#######################################
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.backend_bases import MouseButton
from matplotlib.path import Path


class ManualCluster(object):
    def __init__(self, X_fit):
        
        self.fig = plt.figure()
        self.ax = plt.subplot()

        self.x0 = None
        self.y0 = None
        self.x1 = None
        self.y1 = None
        self.cluster_locs = []
        self.cluster_ids = []
        self.coords = []
        
        
        #
        self.data = X_fit
        
        #
        self.draw()
        
        #self.ax.add_patch(self.rect)
        self.ax.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.ax.figure.canvas.mpl_connect('button_release_event', self.on_release)

    def points_inside_polygon(self):
    
        #FIND points inside polygon
        clustered_ids = []
        clustered_locs = []
        bbPath = Path(np.array(self.coords))
        for k,d in enumerate(self.data):
            if bbPath.contains_point(d):
                clustered_ids.append(k)
                clustered_locs.append(self.data[k])

        # Delete the clustered_pts indexes from 
        # 
        self.data = np.delete(self.data, clustered_ids, axis=0)

        # 
        self.clustered_ids = np.array(clustered_ids)
        
        #
        self.clustered_locs = np.array(clustered_locs)

        #

    def draw(self):
         self.ax.scatter(self.data[:,0], self.data[:,1],
                         c='blue')
        
        
    def on_press(self, event):
        #print ('press')
        
        if event.button is MouseButton.LEFT:
            #print ("LEFT")

            # 
            self.x0 = event.xdata
            self.y0 = event.ydata
            #print ("adding pts: ", self.x0, self.y0)
            self.coords.append([self.x0, self.y0])
            
            # 
            self.ax.scatter([self.x0, self.x0],
                        [self.y0, self.y0],
                        c='black')

        if event.button is MouseButton.RIGHT:
            #print ("RIGHT")
            for k in range(len(self.coords)-1):
                self.ax.plot([self.coords[k][0], self.coords[k+1][0]],
                             [self.coords[k][1], self.coords[k+1][1]],
                        c='black')
            # 
            self.ax.plot([self.coords[0][0], self.coords[-1][0]],
                         [self.coords[0][1], self.coords[-1][1]],
                    c='black')

            
            self.points_inside_polygon()
            
            
            #
            self.cluster_ids.append(self.clustered_ids)
            print ("# of clustered points: ", self.clustered_ids.shape)
            
            # 
            self.cluster_locs.append(self.clustered_locs)
            print ("# of clusters: ", len(self.clustered_locs))
            
            # 
            print ("all_data remaining: ", self.data.shape)
  
            # rset everything
            self.ax.cla()
            self.draw()
            self.coords=[]
            
            
        if event.button is MouseButton.MIDDLE:
            
            print ("SAVING clustering data")
            np.savez('/home/cat/cluster.npz',
                     cluster_locs = self.cluster_locs,
                     cluster_ids = self.cluster_ids)
            
    # 
    def on_release(self, event):
        #print ('release')
        #self.x1 = event.xdata
        #self.y1 = event.ydata
        #self.rect.set_width(self.x1 - self.x0)
        #self.rect.set_height(self.y1 - self.y0)
        #self.rect.set_xy((self.x0, self.y0))
        self.ax.figure.canvas.draw()

a = ManualCluster(X_fit)


release
release
release
release
release
release
release
release
# of clustered points:  (162536,)
# of clusters:  162536
all_data remaining:  (109914, 2)
release
release
release
release
release
release
# of clustered points:  (1217,)
# of clusters:  1217
all_data remaining:  (108697, 2)
release
release
release
release
release
release
# of clustered points:  (4776,)
# of clusters:  4776
all_data remaining:  (103921, 2)
release
release
release
release
release
release
release
# of clustered points:  (2884,)
# of clusters:  2884
all_data remaining:  (101037, 2)
release
release
release
release
release
release
release
# of clustered points:  (4800,)
# of clusters:  4800
all_data remaining:  (96237, 2)
release
release
release
release
release
release
release
release
# of clustered points:  (5385,)
# of clusters:  5385
all_data remaining:  (90852, 2)
release
release
release
release
release
release
# of clustered points:  (3645,)
# of clusters:  3645
all_data remaining:  (87207, 2)
release
relea

/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


release


In [95]:
data = np.load('/home/cat/cluster.npz', allow_pickle=True)
locs = data['cluster_locs']
print ("len ", len(locs))

fig = plt.figure()
ax=plt.subplot(121)
idx = np.random.choice(np.arange(len(locs)), len(locs), replace=False)
for ctr,k in enumerate(idx):
    plt.scatter(locs[k][:,0],
                locs[k][:,1],
                color = 'blue')

ax=plt.subplot(122)

colors = plt.cm.gist_ncar(np.linspace(0,1,len(locs)))
idx = np.random.choice(np.arange(len(locs)), len(locs), replace=False)
for ctr,k in enumerate(idx):
    plt.scatter(locs[k][:,0],
                locs[k][:,1],
                color = colors[ctr])
plt.show()

len  39


In [36]:
from sklearn.cluster import AffinityPropagation
import numpy as np

# 
X1 = X_fit[::5] #[:30*18]
print ("X1: ", X1.shape)

X2 = scipy.spatial.distance.cdist(X1, X1, metric='cosine')
print ("X2: ", X2.shape)

clustering = AffinityPropagation(affinity='precomputed', 
                                 preference=-1.1,
                                 random_state=5, 
                                 verbose=True).fit(X2)

#
print ("FINISHED: ", np.unique(clustering.labels_))

# 
fig = plt.figure()
plt.scatter(X_fit[:,0], X_fit[:,1],
            c='red',
            edgecolor='black',
            alpha=.1)

labels = clustering.labels_
plt.scatter(X1[:,0], X1[:,1],
            c=labels,
            edgecolor='black',
            alpha=1)

plt.show()


X1:  (5449, 2)
X2:  (5449, 5449)
Converged after 96 iterations.
FINISHED:  [0 1 2 3]


In [53]:
print ("X_fit: ", X_fit.shape)

fig = plt.figure()
plt.scatter(X_fit[:,0], X_fit[:,1],
           alpha=.02,
            edgecolor='black'
           )

plt.show()


X_fit:  (272450, 2)


In [159]:
#####################################
print ("all data: ", all_unique.shape)

# 
def cleanup_data(all_unique, min_occurance = 10, max_occurance = 1E4):

    all_clean = all_unique.copy()
    
    frames_unique, indexes, counts_all  = np.unique(all_unique, axis=0, 
                                                return_counts=True, 
                                                return_index=True)
    
    # 
    print ("uinque feature vectors: ", frames_unique.shape, " Counts: ", counts_all.shape, "  all indexes: ", indexes.shape)
    print ("unique features: ", frames_unique)
    print ("counts_all:", counts_all)
    print ("indexes: ", indexes)
    
    # 
    cts_unique = np.unique(counts_all)#[::-1]
    print (" unique counts: ", cts_unique)

    ##################################
    ##################################
    ##################################
    for c in cts_unique:
        idx3 = np.where(counts==c)[0]
        n_counts.append(idx3.shape[0])

        if c<min_occurance or c>max_occurance:
            id4 = np.where(counts_all==c)[0]
            print (" # of unique frames that occur ", c, " times is: ", idx3.shape)
            #print ("ID4: ", id4)
            
            for id_ in id4:
                frame = frames_unique[id_]

                id4 = np.sum(np.equal(all_clean, frame),axis=1)
                id4 = np.where(id4==len(frame))[0]

                all_clean = np.delete(all_clean,id4,axis=0)


    ##################################
    ##################################
    ################################## 
    frames_unique, indexes, counts_all  = np.unique(all_clean, axis=0, 
                                                    return_counts=True, 
                                                    return_index=True)
    
    # 
    print ("uinque feature vectors: ", frames_unique.shape, " Counts: ", counts_all.shape, "  all indexes: ", indexes.shape)
    print ("unique features: ", frames_unique)
    print ("counts_all:", counts_all)
    print ("indexes: ", indexes)
    
    # 
    cts_unique = np.unique(counts_all)#[::-1]
    
    
    
    # 
    return all_clean, cts_unique

##############################################
##############################################
##############################################
min_occurance = 1
max_occurance = 100
all_clean, cts_unique = cleanup_data(all_unique, min_occurance, max_occurance)

print("ALL CLENA: ", all_clean.shape)

plt.plot(cts_unique)
plt.xlabel("Number of frames")
plt.ylabel("Number of occurances")
#plt.xlim(left=-100)
plt.ylim(bottom=1)
plt.semilogy()
plt.semilogx()
plt.show()

all data:  (272450, 7)
uinque feature vectors:  (17355, 7)  Counts:  (17355,)   all indexes:  (17355,)
unique features:  [[0.    0.04  0.96  ... 0.    0.391 0.348]
 [0.    0.04  0.96  ... 0.087 0.565 0.087]
 [0.    0.04  0.96  ... 0.13  0.565 0.043]
 ...
 [0.96  0.04  0.    ... 0.    0.043 0.   ]
 [0.96  0.04  0.    ... 0.043 0.    0.   ]
 [0.96  0.04  0.    ... 0.    0.    0.   ]]
counts_all: [  1   1   1 ... 152 361 725]
indexes:  [267916 127025 267495 ...    312     52   1668]
 unique counts:  [     1      2      3      4      5      6      7      8      9     10
     11     12     13     14     15     16     17     18     19     20
     21     22     23     24     25     26     27     28     29     30
     31     32     33     34     35     36     37     38     39     40
     41     42     43     44     45     46     47     48     49     50
     51     53     54     56     57     58     59     60     61     62
     63     65     66     68     69     70     71     74     75     76
 

 # of unique frames that occur  596  times is:  (1,)
 # of unique frames that occur  602  times is:  (1,)
 # of unique frames that occur  611  times is:  (2,)
 # of unique frames that occur  618  times is:  (1,)
 # of unique frames that occur  623  times is:  (1,)
 # of unique frames that occur  627  times is:  (2,)
 # of unique frames that occur  632  times is:  (1,)
 # of unique frames that occur  664  times is:  (1,)
 # of unique frames that occur  670  times is:  (1,)
 # of unique frames that occur  684  times is:  (1,)
 # of unique frames that occur  692  times is:  (1,)
 # of unique frames that occur  724  times is:  (1,)
 # of unique frames that occur  725  times is:  (1,)
 # of unique frames that occur  734  times is:  (1,)
 # of unique frames that occur  3327  times is:  (1,)
 # of unique frames that occur  162536  times is:  (1,)
uinque feature vectors:  (17181, 7)  Counts:  (17181,)   all indexes:  (17181,)
unique features:  [[0.    0.04  0.96  ... 0.    0.391 0.348]
 [0.   

In [90]:
arg = np.argsort(counts)[::-1]
print (arg)


# print (counts.shape)
# print (counts[::-1])
# plt.plot(np.arange(counts.shape[0]), counts[::-1])
# plt.show()

[ 3294  3290   318 ... 10275 10274     0]


In [78]:
fig = plt.figure()
plt.scatter(X_fit[:,0], X_fit[:,1],
           alpha=.1)

plt.show()

In [ ]:
# EDM CALCULATIONS BELOW

In [7]:
##########################################################
######## COMPUTE EMD ON UNIQUE FEATURE WISE PAIRS ########
########################################################## 
emd_angle_unique_norm, emd_acc_unique_norm = compute_emd_normalized_unique_angle_acc(ang_unique, acc_unique)


100%|██████████| 706/706 [00:06<00:00, 107.06it/s]


In [8]:
###################################
######## GENERATE S MATRIX ########
###################################
#
print (ang_unique.shape)
print (acc_unique.shape)
print (all_unique.shape)      

#     
if True:
    n_split = 200
    indexes_split = np.array_split(np.arange(all_unique.shape[0]),n_split)#[:32]
    parmap.map(compute_S_matrix, indexes_split,
                     all_unique, ang_unique, acc_unique,n_split,
                     pm_processes= 8,
                     pm_pbar = True)
    
else:
    res = []
    for k in range(all_unique.shape[0]):
        res.append(compute_S_matrix([k], all_unique, ang_unique, acc_unique))
        


(325, 3)
(706, 4)
(17355, 7)


203it [23:53,  7.06s/it]                          


(17355, 17355)


/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/matplotlib/image.py:446: UserWarning: Warning: converting a masked element to nan.
  dv = np.float64(self.norm.vmax) - np.float64(self.norm.vmin)
/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/matplotlib/image.py:453: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/matplotlib/image.py:458: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)


In [12]:
####################################################
######## RECONSTRUCT AND SYMMETRIZE S_UNIQUE #######
####################################################

def reconstruct_S_unique(res, all_unique, indexes_split):

    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    fname_out = os.path.join(root_dir, "S_unique_symmetric.npy")
    
    S_unique = np.zeros((all_unique.shape[0],all_unique.shape[0]),'float32') #+np.nan

    for k in range(len(indexes_split)):
        S_unique[indexes_split[k]] = res[k]

    print (S_unique.shape)

    S_unique_symmetric = S_unique + S_unique.T - np.diag(np.diag(S_unique))

    np.save(fname_out, S_unique_symmetric)
    
    return S_unique_symmetric


def reconstruct_S_unique_from_files(all_unique, indexes_split):

    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    fnames = glob.glob(os.path.join(root_dir, "S_unique")+"/*.npz")
    
   
    #S_unique = np.zeros((all_unique.shape[0],all_unique.shape[0]),'float32') #+np.nan
    S_unique = [] #np.zeros((all_unique.shape[0],all_unique.shape[0]),'float32') #+np.nan

    for fname in tqdm(fnames):
        # 
        data = np.load(fname, allow_pickle=True)
        
        # 
        S = data['S']
        indexes = data['indexes']
        
        #
        S_unique[indexes] = S

        
    S_unique_symmetric = S_unique + S_unique.T - np.diag(np.diag(S_unique))

            
    return S_unique_symmetric


# 
S_uniq_sym = reconstruct_S_unique_from_files(all_unique, indexes_split)

    
# plt.imshow(S_uniq_sym, aspect='auto')
# plt.show()

NameError: name 'all_unique' is not defined

In [25]:
#################################
######## CLUSTER S_UNIQUE #######
#################################

from sklearn.cluster import AffinityPropagation
import numpy as np
# X = np.array([[1, 2], [1, 4], [1, 0],
#               [4, 2], [4, 4], [4, 0]])

X = S_uniq_sym #[:30*18]
# print ("X: ", X.shape)
# clustering = AffinityPropagation(random_state=5).fit(X)

# #
#clustering

#
cluster_ids = np.unique(clustering.labels_)
print ("# clusters: ", np.unique(cluster_ids))

# 
S_unique_clustered = np.zeros(X.shape, 'float32')

ctr=0
for c in cluster_ids:
    idx = np.where(clustering.labels_==c)[0]
    print (c,"idx: ", idx.shape)
    S_unique_clustered[ctr:ctr+idx.shape[0]] = S_uniq_sym[idx]
    ctr+=idx.shape[0]

    
# ax2=plt.subplot(121)
# plt.imshow(X, aspect='auto')



# ax2=plt.subplot(122)
# plt.imshow(S_unique_clustered, aspect='auto')
# plt.show()

# 
# clustering.predict([[0, 0], [4, 4]])

# 
#print (clustering.cluster_centers_)


# clusters:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 2

In [58]:
# ###############################################
# ###############################################
# ###############################################


def dim_red3(X,
             method,
             subsample=5):
    
    #
    from sklearn import decomposition
    import sklearn
    

    ###################################
    ###################################
    ###################################
    if method==0:
        pca = decomposition.PCA(n_components=3)

        X_fit = pca.fit_transform(X)
        print (X_fit.shape)

    elif method==1:
        import umap
        umap = umap.UMAP(n_components=3,
                        init='random',
                        random_state=0)

        X_subsampled = X[::subsample]
        print ("fitting umap on" ,X_subsampled.shape)
        umap_ = umap.fit(X_subsampled)

        print ("transforming alldata ", X.shape)
        X_fit = umap_.transform(X)
   
    
    return X_fit


#
X = all_unique
print("X: ", X.shape)
#labels = clustering.labels_
X_fit = dim_red3(X,
                 method=1,
                 subsample=1)

# 
print ("X_fit: ", X_fit.shape)
#print (" DONE: ", X_pca_fit.shape)

X:  (272450, 7)
fitting umap on (272450, 7)


KeyboardInterrupt: 

In [40]:
# ###############################################
# ###############################################
###############################################
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_fit[:,0], X_fit[:,1], X_fit[:,2],
           c = clustering.labels_)
plt.show()


In [22]:
###############################################
###############################################
###############################################

cluster_ids = np.unique(clustering.labels_)
print ("# clusters: ", cluster_ids)

# 
S_unique_clustered = np.zeros(X.shape, 'float32')

ctr=0
for c in cluster_ids:
    idx = np.where(clustering.labels_==c)[0]
    print ("idx: ", idx.shape, idx)
    S_unique_clustered[ctr:ctr+idx.shape[0]] = X[idx]
    ctr+=idx.shape[0]

    
ax2=plt.subplot(121)
plt.imshow(X, aspect='auto')



ax2=plt.subplot(122)
plt.imshow(S_unique_clustered, aspect='auto')
plt.show()


# clusters:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 2

idx:  (93,) [4789 4790 4791 4792 4793 4794 4795 4796 4797 4798 4799 5499 5500 5501
 5502 5503 5504 5505 5506 5507 5508 5509 5510 5511 5709 5710 5711 5712
 5713 5714 5715 5716 6404 6411 6412 6413 6414 6415 6416 6417 6418 6419
 6420 6421 6422 6423 6446 6447 6448 6449 6450 6451 6452 6453 6454 6455
 6456 6457 6458 6459 6460 6461 6462 6463 6493 6494 6495 6496 6497 6585
 6586 6587 6588 6589 6590 6591 6592 6593 6594 6595 6596 6597 6598 6599
 6600 6601 6602 6603 6604 7279 7280 7281 7282]
idx:  (38,) [5561 5562 5563 6519 6520 6521 6522 6523 6524 6525 6526 6527 6528 6529
 6530 6531 6532 6533 6534 6535 6536 6537 6538 6539 6540 6541 6542 6543
 6544 6545 6546 6547 6548 6549 6550 6551 6552 6553]
idx:  (16,) [6430 6468 6555 6556 6557 6558 6559 6560 6561 6606 6607 6608 6647 6648
 7352 7353]
idx:  (43,) [4861 5402 5403 5782 5792 5793 5794 5795 5841 5842 5843 6322 6323 6395
 6610 6611 6612 6613 6614 6615 6651 6652 6653 6654 6655 6656 6657 6696
 6697 6698 7182 7183 7392 7393 7394 7395 7396 7425 7426 7427

idx:  (59,) [12321 12743 12766 12767 12768 12769 12770 12771 12772 12773 12774 12775
 12776 12777 12778 12779 12797 12800 12801 12802 12803 12804 12805 12806
 12807 12808 12809 12810 12838 12839 12840 12841 12842 12843 12844 12845
 12846 12847 12848 12849 12870 12874 12875 12876 12877 12878 12879 12880
 12881 12882 12883 12917 12918 12919 12922 12923 12924 12925 12926]
idx:  (34,) [12306 12325 12326 12327 12328 12329 12727 12728 12748 12780 12781 12782
 12783 12784 12785 12811 12812 12813 12850 12851 12852 12853 12884 12885
 12886 12887 12933 12934 12979 13256 13290 13320 13354 13394]
idx:  (43,) [12755 12757 12759 12760 12761 12762 12763 12764 12765 12790 12791 12792
 12793 12794 12795 12796 12798 12799 12822 12823 12824 12825 12826 12827
 12828 12829 12830 12831 12832 12833 12834 12835 12836 12837 12863 12865
 12866 12867 12868 13301 13303 13332 13333]
idx:  (75,) [ 9459  9794  9796 10036 10037 10038 10039 10411 10412 10413 10414 10415
 10416 10417 10576 10995 10996 10998 10999 11004

In [ ]:
################################
######## PREDICT S_ALL #########
################################





In [ ]:
################################################
######## VISUALIZE S_ALL + CLUSTER IDS #########
################################################





100%|██████████| 706/706 [00:06<00:00, 105.29it/s]


In [79]:
print ("all angles: ", angles_hist.shape)
print ("all accel: ", acc_hist.shape)

print ("Unique angles: ", ang_unique.shape)
print ("Unique acc: ", acc_unique.shape)

print ("Similarity matrix: ", res_angle.shape)
print ('')
# select 10% of the data
angles_sub = angles_hist[::10]
print ("angles_sub : ", angles_sub.shape)

S_ang = np.zeros((angles_sub.shape[0], angles_sub.shape[0]),'float16')
print ("S: ", S_ang.shape)

# 
idx = np.where(np.where(angles_sub==ang_unique[0])[1]==1)[0]
print ("idx: ", idx)


# for k in range(angles_sub.shape[0]):
#     temp = angles_sub[k]
    
    


# s = 0
# for k in trange(ang_unique.shape[0]):

#     # 
#     temp = ang_unique[k]
#     idx3 = np.sum(np.equal(temp, angles_sub),
#                   axis=1)
#     idx_angles = np.where(idx3==3)[0]
  
    

    #
#     # 
#     temp = ang_unique[k]
#     idx3 = np.sum(np.equal(temp, angles_sub),
#                   axis=1)
    
#     idx_angles = np.where(idx3==3)[0]



print ("all sum: ", s)
# plt.imshow(S)
# plt.show()


all angles:  (272450, 3)
all accel:  (272450, 4)
Unique angles:  (325, 3)
Unique acc:  (706, 4)
Similarity matrix:  (325, 325)

angles_sub :  (27245, 3)
S:  (27245, 27245)
idx:  [   33    34    36   410   435   447   453   519   529   535   547   548
   550   554   559   567   569   572   610   655   793   997  1032  1559
  1988  2007  2033  2085  2186  2266  2290  2303  2317  2322  2327  2329
  2332  2339  2354  2358  2391  2416  2447  2464  2466  2471  2479  2483
  2609  2630  2635  2649  2673  2732  2878  2915  2949  3005  3023  3075
  3080  3084  3105  3129  3204  3215  3259  3273  3298  3300  3322  3457
  3459  3554  3595  3602  3607  3628  3631  3649  3658  3661  3677  3688
  3693  3699  3704  3716  3739  3744  3783  3794  3798  3844  4033  4040
  4192  4347  4456  4533  4615  4720  4737  4896  4953  5135  5142  5239
  5268  5523  5576  5581  5729  5786  5808  5812  5839  5846  5857  5861
  5902  5966  5972  5994  6038  6042  6156  6166  6204  6253  6425  6443
  6482  6483  6485 

In [ ]:
##########################################
##########################################
##########################################

# #
# res = hist_rows
# for k in trange(hist_rows.shape[0]):
#     for p in range(k+1, hist_rows.shape[0],1):
#         temp1 = hist_rows[k]
#         temp2 = hist_rows[p]
#         #print ("temp1: ", temp1. shape, temp2.shape)
#         res = scipy.stats.wasserstein_distance(temp1, 
#                                                temp2)
        
        
def edm_parallel(idx, data_in):
    
    res = []
    for i in idx:
        for p in range(i+1, hist_rows.shape[0],1):
            res.append(scipy.stats.wasserstein_distance(hist_rows[i], 
                                                        hist_rows[k]))
    return res
                  
#                       
import parmap
                       
if True:
    idx_split = np.array_split(np.arange(hist_rows.shape[0]),100)
    res = parmap.map(edm_parallel, 
                     idx_split, 
                     hist_rows[:,:3],
                     pm_processes= 8,
                     pm_pbar=True)
    
    
                       

In [343]:
############################################
########### CLUSTER AND VISUALIZE ##########
############################################    



###
idx, labels = cluster_and_visualize(X_pca_fit,
                            X_pca,
                            n_clusters = None, 
                            cluster_id = None)


means:  (272450, 2)
Labels log:  [0.         0.         0.         ... 0.23180157 0.23180157 0.23180157]
Labels norm:  [0.         0.         0.         ... 0.02766821 0.02766821 0.02766821]
labels:  [13.815511  13.815511  13.815511  ...  3.5874352  3.5874352  3.5874352]
labels:  [1.        1.        1.        ... 0.2596672 0.2596672 0.2596672]
Final labels:  [0.  0.  0.  ... 0.7 0.7 0.7]
k:  0.0 (176653,)
k:  0.6 (833,)
k:  0.7 (17860,)
k:  0.8 (63808,)
k:  0.9 (13272,)
k:  1.0 (24,)


In [313]:
# print (labels)
# fig=plt.figure()
# y = np.histogram(labels,bins=np.arange(0,1,0.01))
# plt.plot(y[1][1:],y[0])
# plt.semilogx()
# plt.semilogy()
# plt.show()

[0.         0.         0.         ... 0.02766821 0.02766821 0.02766821]


In [256]:
# ###############################################
# ###############################################
# ###############################################
# X_pca_fit, X_pca = dim_red2(angles_discretized[idx], 
#                             acc_discretized[idx],
#                             method=1,
#                             subsample=2)

# print (" DONE: ", X_pca_fit.shape)

Xpca:  (92161, 23, 2)
Xpca:  (92161, 46)
fitting umap on (46081, 46)
transforming alldata  (92161, 46)


/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


(92161, 2)


[0 1]
PLOTTING...


In [3]:
# ##########################################
# ##########################################
# ##########################################
# animal_ids = np.arange(4)

# vecs_mov_array = []
# # 
# for animal_id in animal_ids:
#     min_duration = 25   # Number of frames

#     #
#     two, six, headnose = get_lengths(animal_id)

#     #
#     _,_,vectors_idx = get_durations(headnose, 0, animal_id, min_duration = min_duration, plotting = False)
#     print (vectors_idx.shape, vectors_idx)


#     feature_ids = np.arange(2)
#     vecs = get_vectors(animal_id, vectors_idx, feature_ids, min_duration)
#     print (str(min_duration)+ " frame long segment array (2features): ", vecs.shape)
#     print (vecs[0].shape)



#     ##################################
#     ##################################
#     ##################################
#     vecs_ego = vectors_to_egocentric(vecs, animal_id)

    
    

#     ##################################
#     ##################################
#     ##################################
#     idx_movement = find_movements(vecs_ego)
#     print ("# of movements; ", idx_movement.shape)
#     vecs_movement = vecs_ego[idx_movement]
    
    
#     if False:
#         vecs_mov_array.append(vecs_movement)
#         continue


#     ##################################
#     ##################################
#     ##################################
#     angles = get_angles(vecs_movement)

#     print (angles.shape)



#     ##################################
#     ##################################
#     ##################################   

#     X_pca = make_pca_data(vecs_movement, angles)


#     np.save('/home/cat/X_pca'+str(animal_id)+'.npy', X_pca)
    
#     X_pca = dim_red(X_pca)

#     # 
#     colors = plt.cm.viridis(np.linspace(0,1,X_pca.shape[0]))

#     fig=plt.figure()
#     plt.scatter(X_pca[:,0],
#                 X_pca[:,1],
#                 alpha=.5,
#                 edgecolor='black',
#                 color=colors
#                )

# plt.show()



d:  (2069710, 6, 2)


  5%|▍         | 13423/272450 [00:00<00:01, 134216.05it/s]

(272450, 2) [[  18576   18601]
 [  18577   18602]
 [  18578   18603]
 ...
 [2058767 2058792]
 [2058768 2058793]
 [2058769 2058794]]
(2069710, 6, 2)


  0%|          | 965/272450 [00:00<00:28, 9648.38it/s]

25 frame long segment array (2features):  (272450, 25, 2, 2)
(25, 2, 2)
vecs:  (272450, 25, 2, 2)


...

(177945, 2) [[   3411    3436]
 [   3412    3437]
 [   3413    3438]
 ...
 [2059837 2059862]
 [2059838 2059863]
 [2059839 2059864]]
(2069710, 6, 2)


  1%|          | 1122/177945 [00:00<00:15, 11211.19it/s]

25 frame long segment array (2features):  (177945, 25, 2, 2)
(25, 2, 2)
vecs:  (177945, 25, 2, 2)


100%|██████████| 177945/177945 [00:15<00:00, 11136.21it/s]


vecs:  (177945, 25, 2, 2)
ctrq:  170813
# of movements;  (2730,)


In [203]:
# ##########################################
# ######### STACKED PCA ANALYSIS############
# ##########################################
# d = []
# lens = []
# lens.append(0)
# for animal_id in animal_ids:
    
#     temp = np.load('/home/cat/X_pca'+str(animal_id)+'.npy')

#     print (temp.shape)
#     lens.append(temp.shape[0])
#     d.append(temp)
    
# X_pca = np.vstack(d)
# print (X_pca.shape)
    
# lens = np.array(lens)
# print ("lens: ", lens)
# X_pca = dim_red(X_pca)

# # 
# colors = plt.cm.viridis(np.linspace(0,1,X_pca.shape[0]))

# colors = ['red','blue','cyan','green']

# fig=plt.figure()
# for k in range(animal_ids.shape[0]):
#     plt.subplot(2,2,k+1)
#     plt.scatter(
#                 X_pca[lens[:k+1].sum():lens[:k+2].sum(),0],
#                 X_pca[lens[:k+1].sum():lens[:k+2].sum(),1],
                
#                 alpha=.5,
#                 edgecolor='black',
#                 color=colors[k]
#                )

# plt.show()

(5717, 75)
(2852, 75)
(5233, 75)
(2730, 75)
(16532, 75)
lens:  [   0 5717 2852 5233 2730]
X_pca:  (16532, 75)
fitting umap
transforming alldata
plotting:  (16532, 2)


d:  (2069710, 6, 2)


  5%|▍         | 12300/272450 [00:00<00:02, 122992.20it/s]

(272450, 2) [[  18576   18601]
 [  18577   18602]
 [  18578   18603]
 ...
 [2058767 2058792]
 [2058768 2058793]
 [2058769 2058794]]
(2069710, 6, 2)


Bar desc:   0%|          | 652/272450 [00:00<00:41, 6516.88it/s]

25 frame long segment array (2features):  (272450, 25, 2, 2)
(25, 2, 2)
vecs:  (272450, 25, 2, 2)


...

(177945, 2) [[   3411    3436]
 [   3412    3437]
 [   3413    3438]
 ...
 [2059837 2059862]
 [2059838 2059863]
 [2059839 2059864]]
(2069710, 6, 2)


Bar desc:   0%|          | 699/177945 [00:00<00:25, 6981.30it/s]

25 frame long segment array (2features):  (177945, 25, 2, 2)
(25, 2, 2)
vecs:  (177945, 25, 2, 2)


Bar desc: 100%|██████████| 177945/177945 [00:26<00:00, 6633.56it/s]

angles:  (177945, 25)


  0%|          | 641/148259 [00:00<00:23, 6401.76it/s]

vecs:  (148259, 50, 2, 2)


100%|██████████| 148259/148259 [00:23<00:00, 6366.92it/s]


vecs:  (148259, 50, 2, 2)
ctrq:  143231
# of movements;  (2018,)


(2018, 50)


In [195]:
# ################################## 
# ######### VISUALIZE ############ 
# ##################################

# # 
# print (vecs_movement.shape)
# f = 1
# colors = plt.cm.jet(np.linspace(0,1,50))
# for f in range(100):
#     plt.subplot(10,10,f+1)
    
#     idx = np.random.choice(np.arange(vecs_movement.shape[0]))
    
#     #for k in range(vecs_movement.shape[1]):\
#     temp = vecs_movement[idx]
#     # print (temp.shape)
#     plt.scatter(temp[:,0,0],
#                 temp[:,0,1],
#                 color=colors
#                )
    
#     mx = np.max(np.abs(temp[:,0,0]))
#     my = np.max(np.abs(temp[:,0,1]))
#     #plt.xticks([])
#     #plt.yticks([])
#     plt.plot([0,0],[-my,my],'--')
#     plt.plot([-mx,mx],[0,0],'--')
    
# plt.show()
        
# 




vecs_nose:  (2018, 50, 2)
(2018, 50, 3)
X_pca:  (2018, 150)
fitting umap
transforming alldata
plotting:  (2018, 2)


In [227]:
# #
# #print ("vecs ego[0]. shape: ", vecs_ego[0].shape)
# temp = vecs_ego[1200]
# print (temp)
# colors = plt.cm.jet(np.linspace(0,1,50))
# for k in range(temp.shape[0]):
#     plt.scatter(temp[k,:,0],
#              temp[k,:,1],
#              color=colors[k],
#                s=np.array([150, 50]),
#                alpha=.8)
    
#     plt.plot(temp[k,:,0],
#              temp[k,:,1],
#              color=colors[k])        
#     #break


NameError: name 'vecs_ego' is not defined

In [193]:
umap_res = X_pca.copy()
print (umap_res.shape)

(223633, 2)


In [56]:
##################################################   
####### GENERATE PAIR-WISE CENTRE DATASETS #######   
################################################## 

# 
cb.parallel = True

# for each recording centre/align every pair of data
#     TODO: make a single file otherwise generating 15 files per recording
#      TODO : OR MAKE A SINGLE STACK OF DATA
cb.centre_and_align_all_pairs()


100%|██████████| 6/6 [03:29<00:00, 34.95s/it]

 DONE Generating 2-point ground truth datasets for imputation


In [193]:
print(y[0])
print (np.cumsum(y[0][::-1]))

[1243944   86107  160024  146467  182678   31808  218682]
[ 218682  250490  433168  579635  739659  825766 2069710]
